In [ ]:
# !pip install

# Import

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error

# CONFIG

In [ ]:
data_dir = '../ABI_Data'
data_load_from_file = False
pickle_path = './dataframe.pkl'
pickle_create = False
pickle_load = False

num_epochs = 50
batch_size = 16
train_proportion = .8
val_proportion = .1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Code

In [ ]:
if data_load_from_file or pickle_create:
    print("Files start")
    files = [f for f in os.listdir(data_dir) if f.endswith('.npz')][:100]
    print("Files done")
    
    records = []
    print("Records start")
    for file in tqdm(files):
        path = os.path.join(data_dir, file)
        try:
            data = np.load(path)
            records.extend([{k: arr[i] for k, arr in data.items()} for i in range(len(next(iter(data.values()))))])
            # print(f"Loaded: {file}")
            del data
        except Exception as e:
            print(f"Failed to load: {file} : {e}")
            pass
    
    print("DF start")
    df = pd.DataFrame(records)
    print("DF done")
    if pickle_create:
        print("Starting pickle")
        df.to_pickle(pickle_path)
        print(f"Pickle saved to: {pickle_path}")
    del files, records
if pickle_load:
    print("Loading Pickle")
    df = pd.read_pickle('dataframe.pkl')
    print("Pickle Loaded")

In [ ]:
### Leave this as backup

# if data_load_from_file or pickle_create:
#     print("Files start")
#     files = [f for f in os.listdir(data_dir) if f.endswith('.npz')][:100]
#     print("Files done")
    
#     records = []
#     print("Records start")
#     for file in tqdm(files):
#         path = os.path.join(data_dir, file)
#         try:
#             data = np.load(path)
#             records.extend([{k: arr[i] for k, arr in data.items()} for i in range(len(next(iter(data.values()))))])
#             # print(f"Loaded: {file}")
#         except Exception as e:
#             print(f"Failed to load: {file} : {e}")
#             pass
    
#     print("DF start")
#     df = pd.DataFrame(records)
#     print("DF done")
#     if pickle_create:
#         print("Starting pickle")
#         df.to_pickle(pickle_path)
#         print(f"Pickle saved to: {pickle_path}")
        
# if pickle_load:
#     print("Loading Pickle")
#     df = pd.read_pickle('dataframe.pkl')
#     print("Pickle Loaded")

In [ ]:
# def print_column_info(df):
#     for col in df.columns:
#         dtype = df[col].dtype
#         sample = df[col].dropna().iloc[0] if not df[col].dropna().empty else "NaN"
#         print(f"{col}: {dtype}, sample: {sample}")
# print_column_info(df)


In [ ]:
# df = df.dropna(subset=['l2_cloud_mask', 'l2_cloud_phase', 'l2_cod', 'l2_cps'])
# print(f"Remaining samples after dropping NaNs: {len(df)}")

In [ ]:
# def preprocess(df):
#     # Inputs: stack rad arrays (N samples, 128 rows, 16 channels)
#     # We want shape (N, channels=16, height=128)
#     rad_np = np.stack(df['rad'].values)  # shape (N, 128, 16)
#     X = torch.tensor(rad_np, dtype=torch.float32).permute(0, 2, 1)  # (N, 16, 128)
    
#     # Targets: stack and convert
#     y_mask = torch.tensor(np.stack(df['l2_cloud_mask'].values), dtype=torch.long)    # (N, 128)
#     y_phase = torch.tensor(np.stack(df['l2_cloud_phase'].values), dtype=torch.long)  # (N, 128)
#     y_cod = torch.tensor(np.stack(df['l2_cod'].values), dtype=torch.float32)         # (N, 128)
#     y_cps = torch.tensor(np.stack(df['l2_cps'].values), dtype=torch.float32)         # (N, 128)
    
#     return X, y_mask, y_phase, y_cod, y_cps

# X, y_mask, y_phase, y_cod, y_cps = preprocess(df)

In [ ]:
class MultiTaskModel(nn.Module):
    def __init__(self, in_channels=16, seq_length=128):
        super().__init__()
        
        # Shared feature extractor
        self.conv1 = nn.Conv1d(in_channels, 64, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(64)
        
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(256)
        
        self.dropout = nn.Dropout(0.3)
        
        # Individual heads with extra layers
        self.mask_head = nn.Sequential(
            nn.Conv1d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(128, 2, kernel_size=1)  # 2 classes: mask / no-mask
        )
        
        self.phase_head = nn.Sequential(
            nn.Conv1d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(128, 5, kernel_size=1)  # 5 classes
        )
        
        self.cod_head = nn.Sequential(
            nn.Conv1d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(128, 1, kernel_size=1)  # Regression
        )
        
        self.cps_head = nn.Sequential(
            nn.Conv1d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(128, 1, kernel_size=1)  # Regression
        )

    def forward(self, x):
        # Shared backbone
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)
    
        # Individual heads
        mask_logits = self.mask_head(x)
        phase_logits = self.phase_head(x)
        cod_pred = self.cod_head(x).squeeze(1)
        cps_pred = self.cps_head(x).squeeze(1)
    
        mask_pred = mask_logits.argmax(dim=1)
    
        mask_filter = (mask_pred == 1).float()
    
        phase_logits = phase_logits * mask_filter.unsqueeze(1)
        cod_pred = cod_pred * mask_filter
        cps_pred = cps_pred * mask_filter
    
        return mask_logits, phase_logits, cod_pred, cps_pred

    # def forward(self, x):
    #     # Shared backbone
    #     x = F.relu(self.bn1(self.conv1(x)))
    #     x = F.relu(self.bn2(self.conv2(x)))
    #     x = F.relu(self.bn3(self.conv3(x)))
    #     x = self.dropout(x)

    #     # Individual heads
    #     mask_logits = self.mask_head(x)
    #     phase_logits = self.phase_head(x)
    #     cod_pred = self.cod_head(x).squeeze(1)
    #     cps_pred = self.cps_head(x).squeeze(1)

    #     return mask_logits, phase_logits, cod_pred, cps_pred        

In [ ]:
# rows_with_zero = df[df['l2_cloud_mask'].apply(lambda x: 0.0 in x)].head(50)
# x = rows_with_zero.iloc[0,:]



# mask = np.array(x['l2_cloud_mask'])
# zero_indices = np.where(mask == 0)[0]
# print(zero_indices)
# mask_zero = phase[zero_indices]

# print(mask_zero)

In [ ]:
# rows_with_zero = df[df['l2_cloud_mask'].apply(lambda x: 0.0 in x)].head(5)

# for idx, row in rows_with_zero.iterrows():
#     # Convert to NumPy arrays (in case they're lists)
#     mask = np.array(row['l2_cloud_mask'])
#     phase = np.array(row['l2_cloud_phase'])
#     cod = np.array(row['l2_cod'])
#     cps = np.array(row['l2_cps'])

#     # Find where mask == 0
#     zero_indices = np.where(mask == 0)[0]

#     # Extract corresponding values
#     mask_zero = phase[zero_indices]
#     phase_zero = phase[zero_indices]
#     cod_zero = cod[zero_indices]
#     cps_zero = cps[zero_indices]

#     print(f"\nRow {idx} — {len(zero_indices)} cloud-free points:")
#     print("  l2_cloud_mask: ", mask_zero)
#     print("  l2_cloud_phase:", phase_zero)
#     print("  l2_cod:        ", cod_zero)
#     print("  l2_cps:        ", cps_zero)

In [ ]:
mask_loss_fn = nn.CrossEntropyLoss()   # expects shape (N, C, L), targets shape (N, L)
phase_loss_fn = nn.CrossEntropyLoss()
cod_loss_fn = nn.MSELoss()
cps_loss_fn = nn.MSELoss()

def multitask_loss(outputs, targets, weights=None):
    mask_logits, phase_logits, cod_pred, cps_pred = outputs
    mask_target, phase_target, cod_target, cps_target = targets
        
    # CrossEntropyLoss expects target shape (N, L) with class indices (long)
    loss_mask = mask_loss_fn(mask_logits, mask_target.long())
    loss_phase = phase_loss_fn(phase_logits, phase_target.long())
    
    # Regression losses: input and target shape (N, L), float
    loss_cod = cod_loss_fn(cod_pred, cod_target)
    loss_cps = cps_loss_fn(cps_pred, cps_target)
    
    # Combine losses with optional weighting
    if weights is None:
        weights = [1.0, 1.0, 1.0, 1.0]
    
    total_loss = weights[0]*loss_mask + weights[1]*loss_phase + weights[2]*loss_cod + weights[3]*loss_cps
    return total_loss, (loss_mask, loss_phase, loss_cod, loss_cps)

def compute_accuracy(logits, targets):
    preds = torch.argmax(logits, dim=1)  # shape: (batch, seq_len)
    correct = (preds == targets).float()
    return correct.mean().item()

def compute_metrics(outputs, targets):
    mask_logits, phase_logits, cod_pred, cps_pred = outputs
    y_mask, y_phase, y_cod, y_cps = targets

    # Get predicted classes
    mask_preds = torch.argmax(mask_logits, dim=1).detach().cpu().numpy()
    phase_preds = torch.argmax(phase_logits, dim=1).detach().cpu().numpy()
    
    # True classes
    y_mask_np = y_mask.detach().cpu().numpy()
    y_phase_np = y_phase.detach().cpu().numpy()
    
    # Classification accuracy
    mask_acc = accuracy_score(y_mask_np.flatten(), mask_preds.flatten())
    phase_acc = accuracy_score(y_phase_np.flatten(), phase_preds.flatten())

    # Regression metrics
    cod_mae = mean_absolute_error(y_cod.detach().cpu().numpy().flatten(), cod_pred.detach().cpu().numpy().flatten())
    cps_mae = mean_absolute_error(y_cps.detach().cpu().numpy().flatten(), cps_pred.detach().cpu().numpy().flatten())

    return mask_acc, phase_acc, cod_mae, cps_mae


In [ ]:
class MultiTaskDataset(Dataset):
    def __init__(self, df):

        self.df = df.reset_index(drop=True)
        
        # # Filter samples where all dqf flags equal 1 inside arrays
        # valid_mask = df['l2_dqf_cloud_mask'].apply(lambda arr: (arr == 1).all())
        # valid_phase = df['l2_dqf_cloud_phase'].apply(lambda arr: (arr == 1).all())
        # valid_cod = df['l2_dqf_cod'].apply(lambda arr: (arr == 1).all())
        # valid_cps = df['l2_dqf_cps'].apply(lambda arr: (arr == 1).all())

        # valid_idx = valid_mask & valid_phase & valid_cod & valid_cps

        # self.df = df[valid_idx].reset_index(drop=True)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Get rad input: shape (128, 16) -> transpose to (16, 128)
        rad = self.df.loc[idx, 'rad'].astype('float32').T  # final shape: (16, 128)
        
        # Targets (all shape (128,))
        mask = self.df.loc[idx, 'l2_cloud_mask'].astype('int64')
        phase = self.df.loc[idx, 'l2_cloud_phase'].astype('int64')
        cod = self.df.loc[idx, 'l2_cod'].astype('float32')
        cps = self.df.loc[idx, 'l2_cps'].astype('float32')
        
        # Convert to torch tensors
        rad_t = torch.tensor(rad)       # shape: (16, 128)
        mask_t = torch.tensor(mask)     # shape: (128,)
        phase_t = torch.tensor(phase)   # shape: (128,)
        cod_t = torch.tensor(cod)       # shape: (128,)
        cps_t = torch.tensor(cps)       # shape: (128,)
        
        return rad_t, mask_t, phase_t, cod_t, cps_t

In [ ]:
dataset = MultiTaskDataset(df)
model = MultiTaskModel()
model.to(device)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_weights = [1.0,1.0,1.0,1.0]

In [ ]:
epoch_losses = []
epoch_mask_acc = []
epoch_phase_acc = []
epoch_cod_mae = []
epoch_cps_mae = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    all_mask_acc = []
    all_phase_acc = []
    all_cod_mae = []
    all_cps_mae = []

    for X, y_mask, y_phase, y_cod, y_cps in loader:
        X, y_mask, y_phase, y_cod, y_cps = X.to(device), y_mask.to(device), y_phase.to(device), y_cod.to(device), y_cps.to(device)

        optimizer.zero_grad()
        outputs = model(X)
        targets = (y_mask, y_phase, y_cod, y_cps)

        loss, _ = multitask_loss(outputs, targets, weights=loss_weights)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Metrics
        mask_acc, phase_acc, cod_mae, cps_mae = compute_metrics(outputs, targets)
        all_mask_acc.append(mask_acc)
        all_phase_acc.append(phase_acc)
        all_cod_mae.append(cod_mae)
        all_cps_mae.append(cps_mae)



    # Epoch summary
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {total_loss:.4f}, "
          f"Mask Acc: {np.mean(all_mask_acc):.4f}, "
          f"Phase Acc: {np.mean(all_phase_acc):.4f}, "
          f"COD MAE: {np.mean(all_cod_mae):.2f}, "
          f"CPS MAE: {np.mean(all_cps_mae):.2f}")
    epoch_losses.append(total_loss)
    epoch_mask_acc.append(np.mean(all_mask_acc))
    epoch_phase_acc.append(np.mean(all_phase_acc))
    epoch_cod_mae.append(np.mean(all_cod_mae))
    epoch_cps_mae.append(np.mean(all_cps_mae))


In [ ]:
epochs = list(range(1, num_epochs + 1))

plt.figure(figsize=(8, 8))

# Accuracy subplot
plt.subplot(2, 1, 1)
plt.plot(epochs, epoch_mask_acc, label='Mask Accuracy')
plt.plot(epochs, epoch_phase_acc, label='Phase Accuracy')
plt.ylim(0, 1)
plt.xlim(1, num_epochs)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Classification Accuracy")
plt.legend()

# MAE subplot
plt.subplot(2, 1, 2)
plt.plot(epochs, epoch_cod_mae, label='COD MAE')
plt.plot(epochs, epoch_cps_mae, label='CPS MAE')
plt.xlim(1, num_epochs)
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.title("Regression MAE")
plt.legend()

plt.tight_layout()
plt.savefig("graphs/Caleb/multi-task-v3.png", dpi=300)
plt.show()
plt.close()
